In [1]:
import os
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import gzip
import pickle as pkl
import pandas as pd

def loadall_results2(path, n_folds):
    regret = []
    history_actions = []
    history_outcomes = []
    perf = []
    with gzip.open(  path ,'rb') as f:
        for i in range(n_folds+1):
            try:
                data = pkl.load(f)
            except EOFError:
                break

            if type(data) == dict:
                regret.append( data['regret'] ) 
                history_actions.append( data['action_history'] )
                history_outcomes.append( data['outcome_history'] ) 
                perf.append( data['pred'] ) 

    return regret, history_actions, history_outcomes, perf

In [25]:
n_folds = 25
horizon = 9999
context = 'MNISTbinary'
model = 'MLP'
case = 'case1'
agent_name = 'EEneuralcbpside_v6' #ineural6, neuronal6

direct = './results/'
path = os.path.join(direct, '{}_{}_{}_{}_{}_{}.pkl.gz'.format(case,model,context,horizon,n_folds,agent_name) )
regret, action_history, outcome_history, perf = loadall_results2(path, n_folds)


In [23]:
def format_perf(perf, model, dataset, l_label):

    processed_data = []
    i = 0
    for item in perf:
        processed_item = {}
        key_mapping = dict( zip( item.keys(), [10, 50, 100, 250, 500, 750, 1000, 2500, 5000, 7500, 9000]))
        new_dict = {key_mapping[old_key]: value for old_key, value in item.items()}
        for key in new_dict:
            # processed_item[f'accuracy_{key}'] = new_dict[key]['accuracy']
            processed_item[f'f1_{key}'] = new_dict[key]['f1']
        processed_item['Seed'] = i
        processed_item['Model'] = model
        processed_item['Dataset'] = dataset
        processed_item['Approach'] = l_label
        i = i+1
        processed_data.append(processed_item)
    return processed_data


def convert_action_history(action_history):
    counters = []
    for history in action_history:
        counter = np.cumsum([ 1 if action == 0 else 0 for action in history ])
        counters.append(counter)

    return np.mean(counters, axis = 0)

n_folds = 26
horizon = 9999
model = 'MLP'

material = {
    #'EEneuralcbpside_v5': {'color': [255, 255, 0], 'label': 'EEneuralcbpside_v5'},  # Red
    'EEneuralcbpside_v6': {'color': [255, 0, 0], 'label': 'Neural-CBP'},  # Red
    'ineural6': {'color': [51, 255, 255], 'label':'IneurAL (official)'},                    # Yellow
    'ineural3': {'color': [0, 0, 255], 'label':'IneurAL (tuned)'},                    # Cyan
    'neuronal6': {'color': [255, 0, 255], 'label':'Neuronal (official)'},                  # Magenta
    'neuronal3': {'color': [160, 160, 160], 'label':'Neuronal (tuned)'},                   # Orange
    # 'margin': {'color': [160, 160, 160], 'label':'Margin'},
    # 'cesa': {'color': [0, 0, 255], 'label':'Cesa'},
}

fig = go.Figure( )

data_models = {}
perfs_final = {}
explo_final = {}

for model in ['MLP']:  # 'LeNet'

    if model == 'MLP':
        n_folds = 25
        datasets = ['MNIST', 'FASHION',  ] # 'covertype', 'shuttle',
    # else:
    #     datasets = ['MNIST', 'FASHION', 'CIFAR10',]
    #     n_folds = 25

    data_regrets = {} 
    data_perfs = {} 
    data_explo = {}
    for data in datasets: 

        # if data in ['MNIST', 'FASHION', 'CIFAR10']:
        #     case = 'case2' 
        # elif data in ['covertype', 'shuttle']:
        #     case = 'game_case_seven'
        # else:
        case = 'case2'
        
        final_regrets = {}
        final_perfs = {}
        explo_perfs = {}
        for agent_name in material.keys():

            color, l_label = material[agent_name]['color'], material[agent_name]['label']

            r,g,b = color

            # try:
            direct = './results/'
            path = os.path.join(direct, '{}_{}_{}_{}_{}_{}.pkl.gz'.format(case, model, data, horizon,n_folds,agent_name) )
            print(path)
            regret, action_history,outcome_history, perf = loadall_results2(path, n_folds)
            regret = np.array(regret)
            regret = regret#[1:]

            final_regrets[l_label] = regret[:,-1] 
            final_perfs[l_label] = pd.DataFrame( format_perf(perf, model, data, l_label) )

            explo_perfs[l_label] = convert_action_history(action_history)

        if data == 'MNISTbinary':
            data = 'MNISTbinary'
        if data == 'FASHION':
            data = 'FASHION'
            
        data_regrets[data] = final_regrets

        dataframes_list = list(final_perfs.values())
        concatenated_df = pd.concat(dataframes_list, axis=0)
        data_perfs[data] = concatenated_df

        data_explo[data] = explo_perfs
        
    data_models[model] = data_regrets
    dataframes_list = list(data_perfs.values())
    concatenated_df = pd.concat(dataframes_list, axis=0)

    explo_final[model] = data_explo

# fig.show()
# fig.write_image("./figures/case1_{}_{}.pdf".format(model, context) )

./results/case2_MLP_MNIST_9999_25_EEneuralcbpside_v6.pkl.gz
./results/case2_MLP_MNIST_9999_25_ineural6.pkl.gz
./results/case2_MLP_MNIST_9999_25_ineural3.pkl.gz
./results/case2_MLP_MNIST_9999_25_neuronal6.pkl.gz
./results/case2_MLP_MNIST_9999_25_neuronal3.pkl.gz
./results/case2_MLP_FASHION_9999_25_EEneuralcbpside_v6.pkl.gz
./results/case2_MLP_FASHION_9999_25_ineural6.pkl.gz
./results/case2_MLP_FASHION_9999_25_ineural3.pkl.gz
./results/case2_MLP_FASHION_9999_25_neuronal6.pkl.gz
./results/case2_MLP_FASHION_9999_25_neuronal3.pkl.gz


In [19]:
data_models['MLP']['Fashion']

{'Neural-CBP': array([2769., 3029., 2834., 2897., 2925., 2935., 2839., 2985., 2856.,
        2864., 2888., 2930., 2901., 2847., 2872., 2890., 2986., 2846.,
        2936., 2865., 2912., 2814., 2909., 3044., 2883.]),
 'IneurAL (official)': array([7968., 8074., 8071., 7606., 7913., 7716., 8005., 8048., 7837.,
        7983., 8127., 7973., 8013., 7788., 8118., 8115., 7985., 7774.,
        7871., 8114., 8446., 8057., 8123., 8019., 8216.]),
 'IneurAL (tuned)': array([4192., 4307., 4296., 4372., 4203., 4406., 4405., 4399., 4342.,
        4359., 4432., 4295., 4387., 4440., 4369., 4370., 4400., 4289.,
        4334., 4257., 4563., 4431., 4456., 4604., 4384.]),
 'Neuronal (official)': array([4835., 4795., 4829., 4901., 4845., 4764., 4839., 4774., 4803.,
        4817., 4886., 4870., 4895., 4843., 4883., 4934., 4950., 4899.,
        4940., 4879., 4965., 4990., 4940., 5055., 4971.]),
 'Neuronal (tuned)': array([3511., 3532., 3581., 3581., 3562., 3533., 3574., 3618., 3542.,
        3711., 3641., 3570.

In [24]:
import pandas as pd
# pd.DataFrame(data_regrets)
# data_regrets

samples = []
# Iterate over the dictionary to extract data
for model, dataset in data_models.items():
    for data, approach in dataset.items():
        for appr, values in approach.items():
            i = 0
            for val in values:
                # print(i, appr,val)
                samples.append([i, model, data, appr, val])
                i = i+1

# Create a DataFrame
df = pd.DataFrame(samples, columns=['Seed','Model', 'Dataset', 'Approach', 'Value'])
df = pd.merge(df, concatenated_df, on=['Model', 'Dataset', 'Approach', 'Seed'])


In [25]:
df[ df['Dataset']=='FASHION' ]

,Seed,Model,Dataset,Approach,Value,f1_10,f1_50,f1_100,f1_250,f1_500,f1_750,f1_1000,f1_2500,f1_5000,f1_7500,f1_9000
125,0,MLP,FASHION,Neural-CBP,2769.0,0.052711,0.313320,0.282629,0.523724,0.670046,0.686633,0.727785,0.760130,0.794938,0.809921,0.822948
126,1,MLP,FASHION,Neural-CBP,3029.0,0.022218,0.155733,0.250111,0.321090,0.568108,0.645585,0.681117,0.748495,0.795881,0.820136,0.825096
127,2,MLP,FASHION,Neural-CBP,2834.0,0.033284,0.353623,0.353583,0.517152,0.665878,0.632297,0.740978,0.764287,0.778509,0.815639,0.813490
128,3,MLP,FASHION,Neural-CBP,2897.0,0.077733,0.362189,0.314126,0.503531,0.666362,0.722280,0.724424,0.760335,0.793495,0.817998,0.828829
129,4,MLP,FASHION,Neural-CBP,2925.0,0.022624,0.362848,0.418842,0.449515,0.671526,0.720255,0.686311,0.758639,0.807916,0.821947,0.822790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,20,MLP,FASHION,Neuronal (tuned),3616.0,0.077795,0.383684,0.463196,0.604995,0.691978,0.735927,0.754805,0.810668,0.840366,0.844609,0.846561
246,21,MLP,FASHION,Neuronal (tuned),3782.0,0.074793,0.226913,0.246994,0.319907,0.377414,0.528712,0.677197,0.792113,0.836275,0.844136,0.842916
247,22,MLP,FASHION,Neuronal (tuned),3511.0,0.046079,0.447282,0.488665,0.601532,0.664465,0.715336,0.743741,0.804592,0.834138,0.840363,0.842056
248,23,MLP,FASHION,Neuronal (tuned),3552.0,0.045356,0.438564,0.491726,0.605147,0.707530,0.741953,0.763232,0.804999,0.837292,0.842933,0.840419


In [26]:
exclude_columns = ['Seed', 'Model', 'Dataset', 'Approach']

agg_columns = [col for col in df.columns if col not in exclude_columns]

df_mean = df.groupby(['Model', 'Dataset', 'Approach'])[agg_columns].agg(['mean',]) # 'std'
df_mean.columns = df_mean.columns.get_level_values(0)
df_mean = df_mean.reset_index()

df_std = df.groupby(['Model', 'Dataset', 'Approach'])[agg_columns].agg(['std',]) # 'std'
df_std.columns = df_std.columns.get_level_values(0)
df_std = df_std.reset_index()


In [27]:
import plotly.graph_objects as go

dataset = 'FASHION'
approaches = [ 'Neuronal (tuned)', 'Neuronal (official)', 'IneurAL (official)', 'IneurAL (tuned)', 'Neural-CBP']
thresholds = ['f1_10', 'f1_50', 'f1_100', 'f1_250', 'f1_500', 'f1_750', 'f1_1000', 'f1_2500', 'f1_5000', 'f1_7500', 'f1_9000']

fig = go.Figure()
for approach in approaches:
    mean_values = [] 
    std_values = []    
    for threshold in thresholds:
        mean_value = df_mean[(df_mean['Dataset'] == dataset) & (df_mean['Approach'] == approach)][threshold].values[0]
        mean_values.append(mean_value)

        std_value = df_std[(df_std['Dataset'] == dataset) & (df_std['Approach'] == approach)][threshold].values[0]
        std_values.append(std_value)

    fig.add_trace(go.Scatter(
        x=[10, 50, 100, 250, 500, 750, 1000, 2500, 5000, 7500, 9000], 
        y=mean_values,
        error_y=dict( type='data',  array=std_values, visible=True),
        line=dict(dash='solid'),
        name=approach  # Sets the name for each trace
    )) 

fig.show()


fig2 = go.Figure()
for approach in approaches:
    mean_values = [] 
    std_values = []    
    
    explo_value = explo_final['MLP'][dataset][approach]
    # print(len(explo_value))

        # std_value = df_std[(df_std['Dataset'] == dataset) & (df_std['Approach'] == approach)][threshold].values[0]
        # std_values.append(std_value)

    fig2.add_trace(go.Scatter(
        x=np.arange(0,horizon,1).tolist(), 
        y=explo_value,
        # error_y=dict( type='data',  array=std_values, visible=True),
        line=dict(dash='solid'),
        name=approach  # Sets the name for each trace
    )) 

fig2.update_yaxes( type="log" )

fig2.show()


In [36]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import plotly.express as px
# Assuming df is your DataFrame and it has columns 'Dataset', 'Value', 'Model'

w = 0.1
def remove_substrings(s, substrings):
    for substring in substrings:
        s = s.replace(substring, '')
    return s


# approaches = result['merge'].unique()
approaches = np.array(['LeNetNeural-CBP', 'LeNetNeuronal (official)', 'LeNetNeuronal (tuned)',
                       'MLPNeural-CBP', 'MLPNeuronal (official)','MLPNeuronal (tuned)', 'MLPIneurAL (official)', 
                       'MLPIneurAL (tuned)',  'MLPCesa', 'MLPMargin' ], dtype=object) 
approaches_names = [ remove_substrings(model, ['LeNet', 'MLP']) for model in approaches ]
default_colors = px.colors.qualitative.Plotly
model_colors = {model_n:col for model_n,col in zip(approaches_names,default_colors) }


# Create an empty figure
fig = go.Figure()


views = []
for appr, appr_name in zip(approaches,approaches_names):

    subset = result[result['merge'] == appr]
    # dec = False
    # if appr_name not in views:
    #     dec = True
    #     views.append(appr_name)
    

    if 'CBP' in appr_name:
        fig.add_trace(go.Bar(
            x=subset['merge2'],
            y=subset['Mean'],
            name=appr_name,
            error_y=dict(type='data', array=subset['StdError'], visible=True,thickness=1 ),
            marker_color=model_colors[appr_name],marker_pattern_shape='x',marker_pattern_fgcolor='black', 
            # showlegend = dec,
            # width=w
            ))
    else: 
        fig.add_trace(go.Bar(
            x=subset['merge2'],
            y=subset['Mean'],
            name=appr_name,
            error_y=dict(type='data', array=subset['StdError'], visible=True,thickness=1 ),
            marker_color=model_colors[appr_name], 
            # showlegend = dec,
            # width=w
            ))


# Add vertical black lines between data groups

datasets = result['merge2'].unique()
print(datasets)
for i, dataset in enumerate(datasets[:-1]):
    if i == 2:
        fig.add_vline(x=i + 0.5, line_width=2, line_color="black")
    else:
        fig.add_vline(x=i + 0.5, line_width=2, line_color="black",  line_dash="dot")


siz = 13

fig.update_layout(
    width=1000,
    height=220,
    plot_bgcolor='white',  # Sets the plot background color
    paper_bgcolor='white',  # Sets the overall figure background color
    barmode='group',
    margin=dict(l=20, r=1, t=25, b=20),  # Small margins
    xaxis=dict(
        tickmode='array',
        tickvals=result['merge2'].unique(),
        ticktext = [ remove_substrings(model, ['LeNet', 'MLP']) for model in result['merge2'].unique() ],
        tickfont=dict(size=siz-2)  # Increase X-axis tick font size
    ),
    yaxis=dict(
        gridcolor='lightgrey',
        title="Final regret ± standard dev.",
        title_standoff=10,
        title_font=dict(size=siz),
        tickfont=dict(size=siz)  # Increase Y-axis tick font size
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.3,  # Adjust this value to position the legend
        xanchor="center",
        x=0.5,
        font=dict(size=siz)  # Increase legend font size
    ),
)

fig.show()
# fig.write_image("./results_MLP.pdf" )

NameError: name 'result' is not defined